In [2]:
!pip install -qU "semantic-router[fastembed]" langchain langchain_community==0.2.6 fastembed==0.3.2 langchain_core openai pymilvus bs4 "grpcio<=1.63.0,>=1.49.1" 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -qU semantic-chunkers==0.0.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.16 requires tiktoken<1,>=0.7, but you have tiktoken 0.6.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [207]:
import requests
from bs4 import BeautifulSoup
import os
import time
from langchain_core.documents import Document
from urllib.parse import urlparse, urljoin
from openai import OpenAI
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, connections, utility
from typing import Any, List, Tuple, Dict, Literal, Optional
from pydantic import Field
from semantic_router.schema import DocumentSplit
from langchain_core.documents import Document
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
from semantic_router.encoders import FastEmbedEncoder
from semantic_router.encoders import OpenAIEncoder
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

In [208]:
## Define the Credentials
ZILLIZ_CLOUD_URI = ""          # os.environ['ZILLIZ_URI'] #"https://in03-c2cc7c5da8decab.api.gcp-us-west1.zillizcloud.com"
ZILLIZ_CLOUD_API_KEY = ""      # os.environ['ZILLIZ_API_KEY']  #"1a8a395165813d72085da55dde1db494f31fc95444ebb4f9e1f9e38127406c2ba82eb29f8b8e3d79c4a06618016e67ac77816ad9"
COLLECTION_NAME=""

In [209]:
DENSE_EMBEDDING_MODEL = "jinaai/jina-embeddings-v2-base-en"
SPARSE_EMBEDDING_MODEL = "Qdrant/bm42-all-minilm-l6-v2-attentions"
SEMANTIC_ENCODER = "jinaai/jina-embeddings-v2-base-en"
#For Rolling Window Technique
SEMANTIC_SCORE_THERESHOLD = 0.3
MIN_SPLII_TOKENS = 100
MAX_SPLIT_TOKENS = 2000

In [210]:
# Check the Milvus connection and collection status First
def connection_status(collection_name):
    connections.connect(
        uri=ZILLIZ_CLOUD_URI,
        token=ZILLIZ_CLOUD_API_KEY
    )

    utility.get_server_version()
    if utility.has_collection(collection_name):
        utility.drop_collection(COLLECTION_NAME)
    else:
        print(f"New Collection -> {collection_name}")

In [211]:
from typing import Any, Dict, List, Optional
from pydantic import BaseModel, Field, model_validator, validator
from langchain_core.embeddings import Embeddings

class SparseFastEmbedEmbeddings(BaseModel, Embeddings):
    """Qdrant FastEmbedding models.
    FastEmbed is a lightweight, fast, Python library built for embedding generation.

    To use this class, you must install the `fastembed` Python package.

    `pip install fastembed`
    Example:
        from langchain_community.embeddings import FastEmbedEmbeddings
        fastembed = FastEmbedEmbeddings()
    """

    model_name: str = "BAAI/bge-small-en-v1.5"
    """Name of the FastEmbedding model to use
    Defaults to "BAAI/bge-small-en-v1.5"
    Find the list of supported models at
    https://qdrant.github.io/fastembed/examples/Supported_Models/
    """

    cache_dir: Optional[str] = Field(default=None)
    """The path to the cache directory.
    Defaults to `local_cache` in the parent directory
    """

    threads: Optional[int] = Field(default=None)
    """The number of threads single onnxruntime session can use.
    Defaults to None
    """

    doc_embed_type: str = "default"
    """Type of embedding to use for documents
    The available options are: "default" and "passage"
    """

    model: Any = Field(default=None, exclude=True)  # Renamed to 'model' and marked as private

    class Config:
        """Configuration for this pydantic object."""
        extra = 'forbid'

    @model_validator(mode='before')
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that FastEmbed has been installed."""
        return values

    def __init__(self, **data):
        super().__init__(**data)
        self._initialize_model()

    def _initialize_model(self):
        """Initialize the FastEmbed model."""
        try:
            # >= v0.2.0
            from fastembed import SparseTextEmbedding

            self.model = SparseTextEmbedding(
                model_name=self.model_name,
                cache_dir=self.cache_dir,
                threads=self.threads,
            )
        except ImportError as ie:
            try:
                # < v0.2.0
                from fastembed.embedding import FlagEmbedding

                self.model = FlagEmbedding(
                    model_name=self.model_name,
                    cache_dir=self.cache_dir,
                    threads=self.threads,
                )
            except ImportError:
                raise ImportError(
                    "Could not import 'fastembed' Python package. "
                    "Please install it with `pip install fastembed`."
                ) from ie

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for documents using FastEmbed.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        embeddings: List[np.ndarray]
        if self.doc_embed_type == "passage":
            embeddings = self.model.passage_embed(texts)
        else:
            embeddings = self.model.embed(texts)
        return [
            {int(idx): float(val) for idx, val in zip(embed.indices, embed.values)}
            for embed in embeddings
        ]

    def embed_query(self, text: str) -> List[float]:
        """Generate query embeddings using FastEmbed.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        query_embeddings: np.ndarray = next(self.model.query_embed(text))
        return query_embeddings.tolist()

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Intelligent WebCrawling

In [213]:
def create_document(content, main_link, section_id, author_name, related_topics, pdf_links):
    doc_data = Document(page_content=content, metadata = {"source_link":main_link, "author_name":author_name, "related_topics":related_topics, "pdf_links":pdf_links})
    return doc_data

def Scrape_data():
    import pandas as pd
    df = pd.read_csv('output_websites_10.csv')
    df = df.fillna("nan")
    documents = []
    for index, row in df.iterrows():
        # Extract data from the DataFrame
        main_link = row['Website']
        content = row['page_content']
        author_name = row['author_name']
        related_topics = row['related_topics']
        pdf_links = row['pdf_links']
        section_id = index  # Or another unique identifier if available
        
        # Create Document instance
        doc_data = create_document(
            content=content,
            main_link=main_link,
            section_id=section_id,
            author_name=author_name,
            related_topics=related_topics,
            pdf_links=pdf_links
        )
        
        # Append to documents list
        documents.append(doc_data)
    return documents

In [214]:
def get_embedding_dim(model_name):
    embeddings = FastEmbedEmbeddings(model_name=model_name)
    document_embeddings = embeddings.embed_documents("Have a great day")
    return len(document_embeddings[0])

In [215]:
def create_collection(collection_name, model_name):
    dimension = get_embedding_dim(model_name)
    fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="source_link", dtype=DataType.VARCHAR, max_length=1000),
        FieldSchema(
            name="text", dtype=DataType.VARCHAR, max_length=65535
        ),
        FieldSchema(
            name="author_name", dtype=DataType.VARCHAR, max_length=65535
            ),
        FieldSchema(
            name="related_topics", dtype=DataType.VARCHAR, max_length=65535
            ),
        FieldSchema(
            name="pdf_links", dtype=DataType.VARCHAR, max_length=65535
        ),
        FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
        FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dimension),
    ]

    schema = CollectionSchema(fields=fields)
    collection = Collection(name=collection_name, schema=schema, shards_num=1, consistency_level="Strong")

    dense_index_params = {
        "index_type": "IVF_SQ8",
        "metric_type": "L2",
        "params": {"nlist": 128},
    }

    sparse_index_params = {
        "index_type": "SPARSE_INVERTED_INDEX",
        "metric_type": "IP",
    }
    collection.create_index(field_name="sparse_vector", index_params=sparse_index_params)
    collection.create_index(field_name="dense_vector", index_params=dense_index_params)
    collection.load()
    return collection

In [216]:
def dense_encode_docs(texts: List[str], embed_model):
    embeddings = FastEmbedEmbeddings(model_name=embed_model)
    document_embeddings = embeddings.embed_documents(texts)
    return document_embeddings

In [217]:
def sparse_encode_docs(texts: List[str], embed_model):
    embeddings = SparseFastEmbedEmbeddings(model_name=embed_model)
    document_embeddings = embeddings.embed_documents(texts)
    return document_embeddings

In [218]:
def embed_insert(data: list, collection, sparse_embed_model, dense_embed_model):
    print(data[1])
    if data[1] == '':
        sparse_embeddings = 0.2
    else:
        sparse_embeddings = sparse_encode_docs(data[1], sparse_embed_model)
    print(sparse_embeddings)
    dense_embeddings = dense_encode_docs(data[1], dense_embed_model)
    collection.insert(
        [
            data[0], # source            
            data[1], # text, page_content
            data[2], # author_name
            data[3], # related_topics
            data[4], #pdf_links
            sparse_embeddings, # sparse_embedding
            dense_embeddings, # dense_embeddings
        ]
    )

In [219]:
def build_metadata(doc):
    doc_metadata = doc.metadata
    source_link_meta = doc_metadata['source_link']
    author_name = doc_metadata['author_name']
    related_topics = doc_metadata['related_topics']
    pdf_links = doc_metadata['pdf_links']
    page_content = doc.page_content
    final_doc = []
    
    metadata = { 
        "author_name": author_name,
        "related_topics": related_topics,
        "source_link": source_link_meta,
        "pdf_links" : pdf_links
    }
    doc_obj = Document(page_content=page_content, metadata=metadata)
    final_doc.append(doc_obj)

    return final_doc

In [220]:
def semantic_splitter(encoder_name, semantic_score_threshold, min_split_tokens, max_split_tokens):
    encoder = FastEmbedEncoder(name=encoder_name)

    encoder.score_threshold = semantic_score_threshold

    splitter = RollingWindowSplitter(
        encoder=encoder,
        dynamic_threshold=False,
        min_split_tokens=min_split_tokens,
        max_split_tokens=max_split_tokens,
        window_size=3,
        plot_splits=False,  # set this to true to visualize chunking
        enable_statistics=True  # to print chunking stats
    )
    return splitter

In [221]:
def insert_data_db(prepared_data, collection_obj, collection_name, sparse_embed_model, dense_embed_model):
    data_batch = [[], [], [], [], []]
    overall_time_st = time.time()
    BATCH_SIZE = 2
    current_source = None
    section = 0
    current_title = None
    doc_title = None

    for content in prepared_data:
        source = content.metadata["source_link"]
        page_content = content.page_content
        author_name = content.metadata["author_name"]
        related_topics = content.metadata["related_topics"]
        pdf_links = content.metadata["pdf_links"]

        data_batch[0].append(source)
        data_batch[1].append(page_content)
        data_batch[2].append(author_name)
        data_batch[3].append(related_topics)
        data_batch[4].append(pdf_links)
        print("Inside Data embed")
        st = time.time()
        ins = embed_insert(data_batch, collection_obj, sparse_embed_model, dense_embed_model)
        print("Total time taken to  process each batch & insert data to milvus is: ", time.time() - st)
        data_batch = [[], [], [], [], []]
        print("="*100)
        # break

    print("overall time to prepare data for insertion: ", time.time() - overall_time_st)

In [222]:
def Store_data_Milvus():
    count = 0
    #Check the Connection status with Milvus
    connection_status(COLLECTION_NAME)

    # Create a collection
    collection = create_collection(COLLECTION_NAME, DENSE_EMBEDDING_MODEL)
    print("Successfully Loaded the Collection")
    
    # Scrape the Base-URL Data
    scraped_data_lst = Scrape_data()

    # Prepare Data
    # splitter = semantic_splitter(SEMANTIC_ENCODER, SEMANTIC_SCORE_THERESHOLD, MIN_SPLII_TOKENS, MAX_SPLIT_TOKENS)

    # Prepare Metadata and Store in Milvus
    for data in scraped_data_lst:
        count+=1
        #splits = splitter([data.page_content])
        #doc_metadata = data.metadata
        prepared_data = build_metadata(data)
        insert_data_db(prepared_data, collection, COLLECTION_NAME, SPARSE_EMBEDDING_MODEL, DENSE_EMBEDDING_MODEL)
    return {f"Successfully Store Data Into the Milvus = {count}"}, collection

In [223]:
# Store The Data Into The Milvus
message, collection = Store_data_Milvus()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 78545.02it/s]


Successfully Loaded the Collection
Inside Data embed
['nan']


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 83330.54it/s]


[{605416826: 0.54747262015521}]


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 56833.39it/s]


...

Total time taken to  process each batch & insert data to milvus is:  1.8365111351013184
overall time to prepare data for insertion:  1.8365843296051025
Inside Data embed
['Sesha was born and brought up in Chennai in a south Indian family. Since childhood, he developed a strong inclination towards finance and numbers, also influenced by his father’s interest in the financial markets. At 15, he surprised his parents, when contrary to everyone’s expectations and wishes of him joining IIT and becoming an engineer, he decided to pursue chartered accountancy. This choice also happened to then become a great motivation for him to push hard and prove himself to his family. In his teens, he spent a lot of time going through CEO statements, analyzing annual reports of companies and understanding and reworking the ratios to increase his understanding and expertise in financial markets.\nReminiscing about how his association with EY began, he recalls his cousin, who is also his mentor in life, enc

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 97921.49it/s]


[{2037567939: 0.16628876822784142, 76510512: 0.0444355718703626, 1788524165: 0.032779594741726384, 1012272114: 0.08850820943323101, 943113075: 0.05487899160863433, 1672556481: 0.06300563691468324, 1853937968: 0.04591179603852249, 874403274: 0.024727138402153417, 1118570297: 0.04131630466850118, 659326392: 0.028009567395102297, 524963102: 0.02694102510665017, 1832758249: 0.03344585180573012, 719013046: 0.029699632404390094, 984398166: 0.0902424187576577, 1308688855: 0.04512775578422707, 1706587157: 0.026617682708933416, 266525280: 0.02961876212054542, 936994131: 0.04941655693535829, 313111706: 0.059121488325604404, 570245443: 0.03614622586811311, 508896879: 0.13068653239386122, 1690623792: 0.09423078072266669, 697303812: 0.06521459391106253, 2090661150: 0.025836694734035697, 319147430: 0.04089420450995254, 655621628: 0.02274122124986989, 534475871: 0.02475677581669569, 1372955919: 0.02388930672660167, 1699170408: 0.023792166221464097, 276369190: 0.03918842231866337, 1858909382: 0.078836

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 78840.30it/s]


Total time taken to  process each batch & insert data to milvus is:  1.8050026893615723
overall time to prepare data for insertion:  1.805077314376831


In [35]:
import pandas as pd
df = pd.read_csv('output_websites.csv')
df['Website'].iloc[0]

'https://www.ey.com/en_in/advanced-manufacturing'

In [59]:
# to check the data from Present in the Milvus
collection_query_source = collection.query(expr='source_link like "https://www.ey.com/en_in/advanced-manufacturing"', output_fields=["text"])

In [51]:
print(1)

1


In [55]:
message

{'Successfully Store Data Into the Milvus = 304'}